As the launch of next generation gaming console around the corner, I think that it is nice to have an overview on the entire gaming market. I also want to have a deep analysis on how well the current generation consoles are doing, namely PS4 vs Xbox One.

This project is for gamers who are interested in gaming on the more popular platform and for developers who want to continue putting resources on current generation consoles. This analysis will offer insights on which platform has stronger player base and which platform sells more copies of games, as well as what kind of impact the current generation consoles could possibly have on the next generation consoles.

#### Part 1: Analysis of gaming market in general

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import seaborn as sns
import sklearn
import pickle
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.utils import column_or_1d
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df1=pd.read_csv('https://raw.githubusercontent.com/dxljack/Gaming-Market-Exploration/master/Dataset/Hardware%20Yearly%20Sales.csv')

In [ ]:
df1

In [ ]:
df2=pd.read_csv('https://raw.githubusercontent.com/dxljack/Gaming-Market-Exploration/master/Dataset/Hardware%20Total%20Sales.csv')

In [ ]:
df2

In [ ]:
df3=pd.read_csv('https://raw.githubusercontent.com/dxljack/Gaming-Market-Exploration/master/Dataset/vgsales-12-4-2019-short.csv')

Because we really only care about Global_Sales, I will replace missing value in Global_Sales with Total_Shipped. Next, I will also replace missing values in Critic_Score with User_Score.

In [ ]:
df3["Global_Sales"] = df3["Global_Sales"].fillna(df3["Total_Shipped"])

In [ ]:
df3["Critic_Score"] = df3["Critic_Score"].fillna(df3["User_Score"])

In [ ]:
# fill in missing values in Global_Sales with mean value of each genre.
df3['Global_Sales'] = df3['Global_Sales'].fillna(df3.groupby('Genre')['Global_Sales'].transform('mean'))

In [ ]:
# fill in missing values in Critic_Score with mean value of each genre.
df3['Critic_Score'] = df3['Critic_Score'].fillna(df3.groupby('Genre')['Critic_Score'].transform('mean'))

In [ ]:
# round numerical values to 2 decimal places
df3 = df3.round(2)

In [ ]:
df3.head()

In [ ]:
df3.isnull().sum()

In [ ]:
df3.describe()

In [ ]:
plt.plot(df1["Year"], df1["PS4"]/1000000, color='blue', label='PS4')
plt.plot(df1["Year"], df1["XOne"]/1000000, color='red', label='XOne')
plt.plot(df1["Year"], df1["NS"]/1000000, color='green', label='NS')
plt.ylabel("Unit sold in millions")
plt.title('Console Yearly Sales')
plt.xticks(df1['Year'])
plt.legend(loc='best')
plt.show()

In [ ]:
plt.bar(df2["Year"], df2["PS4"]/1000000, 0.35, label="PS4", color='lightskyblue')
plt.bar(df2["Year"]+0.35, df2["XOne"]/1000000, 0.35, label="XOne", color='lightcoral')
plt.bar(df2["Year"]+0.35, df2["NS"]/1000000, 0.35, label="NS", color='lightgreen')
plt.ylabel("Unit sold in millions")
plt.title('Console Total Sales')
plt.xticks(df2['Year']+0.35/2, ('2013','2014','2015','2016','2017','2018'))
plt.legend(loc='best')
plt.show()

From early graphs, we can see that in term of sales, PS4 is in the lead compared to XOne and NS.
Next, let us take a good of software in term of game sales, perhaps XOne will take the lead in this round.
Here, NS falls behind is because that it was launched 4 years later.

In [ ]:
mask = (df3["Year"] >= 2013) & (df3["Year"] <= 2018)
df31=df3.loc[mask]
df31.head()

In [ ]:
mask1 = (df31["Platform"] == 'PS4') | (df31["Platform"] == 'XOne') | (df31["Platform"] == 'NS')
df_3 = df31.loc[mask1]

In [ ]:
df_3.head()

Extract the columns we need for further exploration.

In [ ]:
df_3=df_3[["Name", "Genre", "Platform", "Publisher", "Global_Sales", "Year"]]

In [ ]:
df_3

In [ ]:
df_3.isnull().sum()

Let us group the remaining data by Platform and compute the sum of sales.

In [ ]:
df_31=df_3[["Platform", "Global_Sales"]].groupby("Platform").sum()

In [ ]:
df_31.head()

Above is the Global_Sales from 2013 to 2018.

In [ ]:
ax = sns.barplot(df_31.index, df_31["Global_Sales"])
ax.set(xlabel='Platform', ylabel='Global sales in millions')

If you would like to see the yearly breakdown, here it is.

In [ ]:
df_32=df_3.groupby(["Platform", "Year"])["Global_Sales"].sum()

In [ ]:
df_32=df_32.to_frame()
df_32

In [ ]:
ax1 = df_32.unstack().plot(figsize=(8,6), kind='bar')
ax1.legend(["2013","2014","2015","2016","2017","2018"])
ax1.set_ylabel("Global sales in millions")

At this point, we can see that XOne isn't really doing all the well for this generation. It falls behind PS4 every year. Now let us take a look at the exact market share in term of game sales. Note that we cannot explore the market share with PC, since we didnot take into account the PC sales.

In [ ]:
df_33=df_31/df_31["Global_Sales"].sum()
df_33

In [ ]:
labels = 'NS', 'PS4', 'XOne'
sizes = df_33["Global_Sales"]
colors = ['lightgreen', 'lightskyblue', 'lightcoral']
explode = (0, 0.05, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

ax2 = plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.show()

In summary, as far as current generation console goes, the winner is clearly the PS4. This will give a lot of confidence to the developers who will continue to make games for PS4 as time goes on, because even when the next generation launches, PS4 will still be supported many years to come. In addition, this will possibly promotes the sales of PS5 since the player base is high, it is very likely for those players to stay with PlayStation due to few main factors, including previous investments in games and friends in the same network, etc. This concludes the analysis on gaming market.

#### Part 2: Modeling and prediction on future game sales

We can start by finding what kind of game is most popular through the year. For simplicity, we will continue using df_3 which consists of games from 2013 to 2018.

In [ ]:
df3.groupby('Genre')['Critic_Score'].mean()

In [ ]:
df3.isnull().sum()

In [ ]:
df_model = df3[["Name", "Platform", "Genre", "Publisher", "Critic_Score", "Global_Sales", "Year"]]

In [ ]:
df_model.head()

In [ ]:
df_model.isnull().sum()

In [ ]:
top5_genre=df_model.groupby(['Genre'])['Global_Sales'].sum().sort_values(ascending=False).head(5)
top5_genre

In [ ]:
top5_genre=top5_genre.to_frame()
top5_genre_df = df_model[df_model["Genre"].isin(top5_genre.index)]
top5_genre_df.head()

In [ ]:
top20_name=df_model.groupby(['Name'])['Global_Sales'].sum().sort_values(ascending=False).head(20)
top20_name

In [ ]:
top20_name=top20_name.to_frame()
top20_name.head()

In [ ]:
fig, ax2 = plt.subplots(figsize=(15,7))
plt.barh(top20_name.index, top20_name["Global_Sales"], align='center')
plt.xlabel('Global sales in millions')
ax2.invert_yaxis()
plt.show()

In [ ]:
fig, ax3 = plt.subplots(figsize=(15,7))
sns.lineplot(x='Year', y='Global_Sales', hue='Genre', data=top5_genre_df, ci=None, ax=ax3)

In [ ]:
top5_publisher=df_model.groupby(['Publisher'])['Global_Sales'].sum().sort_values(ascending=False).head(5)
top5_publisher

In [ ]:
top5_publisher=top5_publisher.to_frame()
top5_publisher_df = df_model[df_model["Publisher"].isin(top5_publisher.index)]
top5_publisher_df.head()

In [ ]:
fig, ax4 = plt.subplots(figsize=(15,7))
sns.lineplot(x='Year', y='Global_Sales', hue='Publisher', data=top5_publisher_df, ci=None, ax=ax4)

Now that we know what kind of games are popular, we can proceed building a linear model to predict sales with selected features, namely Genre and Publisher.

In [ ]:
df_model = df_model[["Platform", "Genre", "Publisher", "Critic_Score", "Global_Sales"]]
df_model.head()

In [ ]:
df_model.describe()

In [ ]:
X = df_model[['Platform', 'Genre', 'Publisher', 'Critic_Score']]
y = df_model['Global_Sales']

In [ ]:
categorical_columns = ['Platform', 'Genre', 'Publisher']
numerical_columns = ['Critic_Score']

In [ ]:
features = ColumnTransformer([('categorical', OneHotEncoder(), categorical_columns),
                              ('numerical', StandardScaler(), numerical_columns)])

In [ ]:
# Main model for interactive webapp
# Linear regression without cross-validation
lr = LinearRegression()
pipe_lr = Pipeline([
    ('features', features),
    ('estimator', lr)
])

pipe_lr.fit(X, y)
y_pred_lr = pipe_lr.predict(X)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y, y_pred_lr))
# The r2 score
print('R2 Score: %.2f'
      % r2_score(y, y_pred_lr))

In [ ]:
# Next let's try Ridge regression without cross-validation
ridge = Ridge()
pipe_ridge = Pipeline([
    ('features', features),
    ('estimator', ridge)
])

pipe_ridge.fit(X, y)
y_pred_ridge = pipe_ridge.predict(X)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y, y_pred_ridge))
# The r2 score
print('R2 Score: %.2f'
      % r2_score(y, y_pred_ridge))

In [ ]:
# Ridge regression with cross-validation
X_new = features.fit_transform(X,y)
gs_ridge = GridSearchCV(
                ridge,
                {"alpha": np.arange(0, 15, 0.1)},  # range of hyperparameters to test
                cv=10,  # 10-fold cross validation
                n_jobs=2,  # run each hyperparameter in one of two parallel jobs
)
gs_ridge.fit(X_new, y)
y_pred_ridge_gs = gs_ridge.predict(X_new)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y, y_pred_ridge_gs))
# The r2 score
print('R2 Score: %.2f'
      % r2_score(y, y_pred_ridge_gs))

In [ ]:
gs_ridge.best_params_

In [ ]:
y_pred_ridge_gs

In [ ]:
# RandomForest regression without cross-validation
rf = RandomForestRegressor()
pipe_rf = Pipeline([
    ('features', features),
    ('estimator', rf)
])

pipe_rf.fit(X, y)
y_pred_rf = pipe_rf.predict(X)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y, y_pred_rf))
# The r2 score
print('R2 Score: %.2f'
      % r2_score(y, y_pred_rf))

In [ ]:
# Testing model prediction by user input
data = pd.DataFrame([('PS4', 'Shooter', 'Activision', 8)], columns = ['Platform' , 'Genre', 'Publisher' , 'Critic_Score'])

In [ ]:
data

In [ ]:
# Save predictive_model
pickle.dump(pipe_rf, open('predictive_model.sav', 'wb'))

In [ ]:
pipe_rf = pickle.load(open('predictive_model.sav', 'rb'))

In [ ]:
# Results in millions
pipe_rf.predict(data)

In [ ]:
# RandomForest regression with cross-validation
X_new = features.fit_transform(X,y)
gs_rf = GridSearchCV(
                rf,
                {"max_depth": [50,60,70]},  # range of hyperparameters to test
                cv=10,  # 10-fold cross validation
                n_jobs=2,  # run each hyperparameter in one of two parallel jobs
)
gs_rf.fit(X_new, y)
y_pred_rf_gs = gs_rf.predict(X_new)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y, y_pred_rf_gs))
# The r2 score
print('R2 Score: %.2f'
      % r2_score(y, y_pred_rf_gs))

In [ ]:
gs_rf.best_params_

In [ ]:
# Save best_predictive_model
pickle.dump(gs_rf, open('best_predictive_model.sav', 'wb'))

In [ ]:
gs_rf = pickle.load(open('best_predictive_model.sav', 'rb'))

From LinearRegression, RidgeRegression to RandomForestRegressor, there is a noticeable improvement in model performance. Since the results from RandomForestRegressor with or without cross-validation are the same, I will simply use the former as my predictive model. Some additional thoughts, instead of using OneHotEncoding, What if I use Label Encoding with respect to certain hiearchy.

In [ ]:
# df_platform = df_model.groupby(['Platform'])['Global_Sales'].sum().sort_values().index

In [ ]:
# df_genre = df_model.groupby(['Genre'])['Global_Sales'].sum().sort_values().index

In [ ]:
# df_publisher = df_model.groupby(['Publisher'])['Global_Sales'].sum().sort_values().index

In [ ]:
# # LabelEncoding with respect to chosen order
# class LabelEncoder(LabelEncoder):

#     def fit(self, y):
#         y = column_or_1d(y, warn=True)
#         self.classes_ = pd.Series(y).unique()
#         return self

In [ ]:
# le = LabelEncoder()

In [ ]:
# le_platform = le.fit(df_platform)
# le_platform.classes_

In [ ]:
# c1 = le_platform.transform(df_model['Platform'])
# df_model_new = pd.DataFrame()
# df_model_new['Platform'] = c1

In [ ]:
# le_genre = le.fit(df_genre)
# le_genre.classes_

In [ ]:
# c2 = le_genre.transform(df_model['Genre'])
# df_model_new['Genre'] = c2

In [ ]:
# le_publisher = le.fit(df_publisher)
# le_publisher.classes_

In [ ]:
# c3 = le_publisher.transform(df_model['Publisher'])
# df_model_new['Publisher'] = c3

In [ ]:
# df_model_new['Critic_Score'] = df_model['Critic_Score']
# df_model_new['Global_Sales'] = df_model['Global_Sales']

In [ ]:
# # This is the final dataframe for modeling
# df_model_new.head()

In [ ]:
# X_model = df_model_new.loc[:, : 'Critic_Score']
# X_model = StandardScaler().fit_transform(X_model)

In [ ]:
# X_model

In [ ]:
# y_model = df_model_new['Global_Sales']

In [ ]:
# y_model.head()

In [ ]:
# # New Linear regression without train-test-split and cross-validation
# lr_est = LinearRegression()
# lr_est.fit(X_model, y_model)
# y_pred_lr_new = lr_est.predict(X_model)
# # The mean squared error
# print('Mean squared error: %.2f'
#       % mean_squared_error(y_model, y_pred_lr_new))
# # The r2 score
# print('R2 Score: %.2f'
#       % r2_score(y_model, y_pred_lr_new))

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.25, random_state=42)
# lr_est.fit(X_train, y_train)
# y_pred_lr_new_split = lr_est.predict(X_test)
# # The mean squared error
# print('Mean squared error: %.2f'
#       % mean_squared_error(y_test, y_pred_lr_new_split))
# # The r2 score
# print('R2 Score: %.2f'
#       % r2_score(y_test, y_pred_lr_new_split))

In [ ]:
# # New Ridge regression with train-split-test and cross-validation
# gs_ridge_new = GridSearchCV(
#                 ridge,
#                 {"alpha": np.arange(0, 15, 0.1)},  # range of hyperparameters to test
#                 cv=10,  # 10-fold cross validation
#                 n_jobs=2,  # run each hyperparameter in one of two parallel jobs
# )
# gs_ridge_new.fit(X_train, y_train)
# y_pred_ridge_gs_new = gs_ridge_new.predict(X_test)
# # The mean squared error
# print('Mean squared error: %.2f'
#       % mean_squared_error(y_test, y_pred_ridge_gs_new))
# # The r2 score
# print('R2 Score: %.2f'
#       % r2_score(y_test, y_pred_ridge_gs_new))

In [ ]:
# gs_ridge_new.best_params_

In summary, we have successfully predicted game global sales based on platform, genre, publisher and critic_score. As you can see above, the MSE is low as we would prefer. Ideally, top genre games and top publisher games will have bigger sales as well as popular platform games. I'm sure there is room for improvement in the model, this is yet to be explored. For now, this concludes the modeling and predicting.

#### Part 3. Building a gamne recommender system


Now that we are done with business analytics, let's build a game recommender system for us consumers.

In [ ]:
# df4=pd.read_csv('https://raw.githubusercontent.com/dxljack/Gaming-Market-Exploration/master/Dataset/game%20descriptions.csv')

In [ ]:
# df4 = df4[['name', 'desc_snippet']]

In [ ]:
# df4['name'] = df4['name'].str.upper() 

In [ ]:
# # remove all non-ASCII characters
# df4['name'] = df4['name'].str.encode('ascii', 'ignore').str.decode('ascii')

In [ ]:
# df4.head()

In [ ]:
# df4.info()

In [ ]:
mask2 = (df3["Platform"] == 'PS4') | (df3["Platform"] == 'XOne') | (df3["Platform"] == 'NS') | (df3["Platform"] == 'PC')
df_rec = df3.loc[mask2]

In [ ]:
df_rec = df_rec[['Name', 'Platform', 'Genre', 'Publisher', 'Critic_Score', 'Year']]

In [ ]:
df_rec.isnull().sum()

In [ ]:
df_rec = df_rec.dropna(subset=['Year'])

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import nltk

In [ ]:
df_rec['Critic_Score'] = df_rec['Critic_Score'].astype(int)
df_rec['Year'] = df_rec['Year'].astype(int)

In [ ]:
df_rec['Critic_Score'] = df_rec['Critic_Score'].astype(str)
df_rec['Year'] = df_rec['Year'].astype(str)

In [ ]:
df_rec['Name'] = df_rec['Name'].str.upper() 

In [ ]:
# Remove quotations in game names
df_rec['Name'] = df_rec['Name'].str.replace('"','')

In [ ]:
df_rec['Joined Column'] = df_rec[['Name', 'Genre', 'Publisher']].apply(lambda x: ', '.join(x), axis = 1)

In [ ]:
df_rec.head()

In [ ]:
# Save df_rec
df_rec.to_pickle('recommender dataframe') 

In [ ]:
df_rec = pd.read_pickle('recommender dataframe')

In [ ]:
# cv = CountVectorizer()
# cv_matrix = cv.fit_transform(df_rec['Joined Column'])
# cv_name = cv.get_feature_names()
# cv_event = matrix.toarray()

In [ ]:
# cosine_sim = cosine_similarity(cv_matrix, cv_matrix)
# cosine_sim

In [ ]:
# Compute jaccard similarity
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
# Recommend top 5 games
def get_recommendation(name, platform):
    df_rec1 = df_rec[['Name', 'Platform', 'Joined Column']]
    df_rec_final = df_rec1[df_rec1['Platform'] == platform]
    df_rec_final = df_rec_final[['Name', 'Joined Column']]
    game_desc = df_rec_final.groupby('Name')['Joined Column'].apply(list).to_dict() # Convert dataframe to dictionary
    rec_dict = {}
    for k in game_desc.keys():
        rec_dict[k] = get_jaccard_sim(game_desc[name][0], game_desc[k][0])
    sort_dict = sorted(rec_dict.items(), key=lambda x: x[1], reverse=True)
    return [tuple[0] for tuple in sort_dict[1:6]]

In [ ]:
get_recommendation('GRAND THEFT AUTO V', 'PC')

In [ ]:
# # combine rows with same game name and join strings
# df_rec['Platform'] = df_rec.groupby(['Name', 'Genre'])['Platform'].transform(lambda x : ', '.join(x))

In [ ]:
# df_rec = df_rec.drop_duplicates('Name')

In [ ]:
# df_4_rec = df_rec.merge(df4, how='outer', left_on='Name', right_on='name')

In [ ]:
# df_4_rec = df_4_rec[['Name', 'Platform', 'Genre', 'Publisher', 'Critic_Score', 'Year', 'desc_snippet']]

In [ ]:
# df_4_rec = df_4_rec.astype(str)

In [ ]:
# df_4_rec = df_4_rec[df_4_rec['Name'] !='nan']
# df_4_rec = df_4_rec[df_4_rec['desc_snippet'] !='nan']

In [ ]:
# df_4_rec.info()

In [ ]:
# # convert Critic_Score and Year into string 
# # Combine columns into one using join
# df_4_rec['Joined Column'] = df_4_rec[['Platform', 'Genre', 'Publisher', 'Year', 'desc_snippet']].apply(lambda x: ', '.join(x), axis = 1)

In [ ]:
# df_4_rec.head()

In [ ]:
# df_4_rec.info()

In [ ]:
# df_4_rec['Name']

In [ ]:
# tfv = TfidfVectorizer()
# tfv_matrix = tfv.fit_transform(df_4_rec['Joined Column'])

In [ ]:
# tfv_matrix.shape

In [ ]:
# cosine_sim = cosine_similarity(tfv_matrix, tfv_matrix)

In [ ]:
# cosine_sim.shape

In [ ]:
# #Construct a reverse map of indices and game names
# indices = pd.Series(df_4_rec.index, index=df_4_rec['Name']).drop_duplicates()

In [ ]:
# indices[:10]

In [ ]:
# # Defining the function that takes in game name as input and returns the top 5 recommended games
# def get_recommendations(name, cosine_sim = cosine_sim):
    
#     # Get the index of the game that matches the name
#     idx = indices[name]

#     # Get the pairwsie similarity scores of all games with that game
#     sim_scores = list(enumerate(cosine_sim[idx]))

#     # Sort the games based on the similarity scores
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Get the scores of the 5 most similar games
#     sim_scores = sim_scores[1:11]

#     # Get the game indices
#     game_indices = [i[0] for i in sim_scores]

#     # Return the top 5 recommended games
#     return df_4_rec['Name'].iloc[game_indices]

In [ ]:
# get_recommendations("CALL OF DUTY: WWII")